In [9]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import pandas as pd
import os
import sys
from dateutil import parser
from time import time

%matplotlib inline

import sys
sys.path.append('/home/tony/code/python/notebooks/MyLib/EnsembleMachine')

import date_feature_extractor
reload(date_feature_extractor)
from date_feature_extractor import DateFeatureExtractor

import categorical_feature_extractor
reload(categorical_feature_extractor)
from categorical_feature_extractor import CategoricalFeatureExtraction

import train_test_data_loader
reload(train_test_data_loader)
from train_test_data_loader import TrainTestDataLoader

import feature_selection
reload(feature_selection)
from feature_selection import FeatureSelection

#for forward/backward regression
from sklearn.linear_model import LinearRegression, LogisticRegression, LogisticRegressionCV

In [2]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.info("logger loaded!")

INFO:root:logger loaded!


In [3]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
store_df = pd.read_csv('store.csv').drop(['PromoInterval'], axis=1, inplace=True)

/home/tony/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


# Loading the data from the file

In [4]:
t0 = time()
dataLoader = TrainTestDataLoader('train.csv', 'test.csv', train_labels_column='Sales', test_ids_column='Id')
dataLoader.cleanData(max_onehot_limit=200)
print('completed in {0} seconds!'.format(time()-t0))

INFO:train_test_data_loader:reading train file train.csv
INFO:train_test_data_loader:reading test file test.csv
INFO:train_test_data_loader:extracting train labels from train_df[Sales]
INFO:root:loading Date as a date
INFO:root:loading StateHoliday as one-hot (with 5 categories)


completed in 24.1726999283 seconds!


In [5]:
def loadData(train_df, test_df, id_col='Id', label_col='Sales'):

    train_labels = train_df[label_col].values
    train_df.drop(label_col, axis=1, inplace=True)

    test_ids = test_df[id_col].values
    test_df.drop(id_col, axis=1, inplace=True)

    # Convert 'object' columns to one-hot encoding
    # Ordinal encoding is also available for 'object/int/float' columns.
    print('loading one-hot categories ["StateHoliday"]')
    categoricalFeatureExtractor = CategoricalFeatureExtraction()
    categoricalFeatureExtractor.convertColumnsToOneHot([train_df, test_df], ['StateHoliday'])

    #Convert date columns to date features (year, month, day, day-of-week, etc.)
    print('loading date features for "Date"')
    dateFeatureExtractor = DateFeatureExtractor()
    train_df = dateFeatureExtractor.createFeaturesFromDateColumns(train_df, ['Date'])
    test_df = dateFeatureExtractor.createFeaturesFromDateColumns(test_df, ['Date'])

    print('imputing null values with 0s')
    train_df = train_df.fillna(0)
    test_df = test_df.fillna(0)

    categoricalFeatureExtractor = CategoricalFeatureExtraction()
    categoricalFeatureExtractor.removeUnsharedColumns([train_df, test_df])

    #os.mkdir('autoML_rossmann')
    train_df.to_csv('rossmann_clean_train.csv', index=False)
    test_df.to_csv('rossmann_clean_test.csv', index=False)
    pd.DataFrame({'y': train_labels}).to_csv('rossmann_clean_trainLabels.csv', index=False)
    pd.DataFrame({'id': test_ids}).to_csv('rossmann_clean_testIds', index=False)

    return train_df, test_df, train_labels, test_ids

#train_df, test_df, train_labels, test_ids = loadData(train_df, test_df, id_col='Id', label_col='Sales')

In [6]:
X = dataLoader.train_df.values
X_sub = dataLoader.test_df.values
y = dataLoader.train_labels
X.shape, X_sub.shape, y.shape

((1017209, 14), (41088, 14), (1017209,))

# Selecting features

In [11]:
print "Extracting features"
feature_selection = FeatureSelection(lower_is_better=True, method='all', X=X, y=y, clf=LogisticRegressionCV(), problem_type='classification')

Extracting features


In [12]:
X_all = feature_selection.allSelection(X)
X_all.shape, X.shape

((1017209, 14), (1017209, 14))

In [ ]:
X_forwards = FeatureSelection.forwards(X, y, )

In [496]:
z = list(range(10))
z.remove(3)
z

[0, 1, 2, 4, 5, 6, 7, 8, 9]